In [1]:
from utils import reddit
from utils.azure import authenticate_client
from datetime import datetime
import time


In [2]:
comments = reddit.get_reddit_comments("https://old.reddit.com/r/soccer/comments/t820zv/match_thread_manchester_city_vs_manchester_united/")

In [3]:
match_start = time.mktime(datetime(2022,3,6,16,30).timetuple())
match_end = time.mktime(datetime(2022,3,6,18,24).timetuple())

#create time stamps to bucket comments into 5 minute chunks
time_buckets = (list(range(int(match_start), int(match_end), 5*60)))

can see some negative sentiment towards martin tyler here - lets see if this is detected by the language model

In [4]:
first_5_min = [comment for comment in comments if time_buckets[0]< comment[0] < time_buckets[1]]
first_5_min.sort(key= lambda x:x[0])
first_5_min


[(1646584222.0,
  'At least City fans didn’t start singing their owner’s name there.'),
 (1646584270.0, "Martin Tyler sounds like he's dying on the mic"),
 (1646584272.0, 'Man flu on commentary'),
 (1646584311.0, 'Martin Tyler man. The worlds best cure for insomnia'),
 (1646584364.0,
  'Is McFred an official term now? Why am I hearing it on commentary lol'),
 (1646584433.0,
  'My friend is going home to Ukraine to fight. Makes these shows of solidarity hit home a lot more.'),
 (1646584491.0,
  'The weird double affect on the advertising hoardings is confusing the hell out of me.')]

In [59]:
all_comments = [comment[1] for comment in first_5_min]

all_comments_str = ". ".join(all_comments)

In [61]:
client = authenticate_client()
sentiments = client.analyze_sentiment([all_comments_str], show_opinion_mining=True)

overall sentiment

In [62]:
sentiments[0].confidence_scores

SentimentConfidenceScores(positive=0.29, neutral=0.16, negative=0.56)

In [63]:
sentiments[0].sentences[0].confidence_scores

SentimentConfidenceScores(positive=0.03, neutral=0.44, negative=0.53)

In [64]:
opinion_sentences= [sentence for sentence in sentiments[0].sentences if sentence.mined_opinions]

In [51]:
sentence = opinion_sentences[0]
print(f"Text:{sentence.text}\n Opinion: {sentence.mined_opinions[0].assessments[0].text} {sentence.mined_opinions[0].target.text} ")

Text:The worlds best cure for insomnia. 
 Opinion: best cure 
